In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup

In [2]:
# create urls for all seasons of all leagues
base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
seasons = [ '2015', '2016', '2017', '2018', '2019']

In [3]:
# Starting with latest data for Spanish league
url = base_url+'/'+leagues[0]+'/'+seasons[4]
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

# Based on the structure of the webpage, the data is in the JSON variable, under <script> tags
scripts = soup.find_all('script')

### Working with JSON

We found that the data interesting us is stored in teamsData variable, after creating a soup of html tags it becomes just a string, so we find that text and extract JSON from it.

In [4]:
import json

string_with_json_obj = ''

# Find data for teams
for el in scripts:
    if 'teamsData' in el.text:
      string_with_json_obj = el.text.strip()
      
# print(string_with_json_obj)

# strip unnecessary symbols and get only JSON data
ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

Once we have gotten our JSON and cleaned it up we can convert it into Python dictionary and check how it looks.

### Understanding data with Python

In [5]:
# convert JSON data into Python dictionary
data = json.loads(json_data)
print(data.keys())
print('='*50)
print(data['138'].keys())
print('='*50)
print(data['138']['id'])
print('='*50)
print(data['138']['title'])
print('='*50)
print(data['138']['history'][0])

# Print pretty JSON data to check out what we have there
# s = json.dumps(data, indent=4, sort_keys=True)
# print(s)

dict_keys(['138', '140', '141', '142', '143', '146', '147', '148', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '231', '239'])
dict_keys(['id', 'title', 'history'])
138
Sevilla
{'h_a': 'a', 'xG': 2.40044, 'xGA': 0.145879, 'npxG': 2.40044, 'npxGA': 0.145879, 'ppda': {'att': 256, 'def': 23}, 'ppda_allowed': {'att': 327, 'def': 29}, 'deep': 6, 'deep_allowed': 0, 'scored': 2, 'missed': 0, 'xpts': 2.9074, 'result': 'w', 'date': '2019-08-18 18:00:00', 'wins': 1, 'draws': 0, 'loses': 0, 'pts': 3, 'npxGD': 2.2545610000000003}


In [6]:
# Get teams and their relevant ids and put them into separate dictionary
teams = {}
for id in data.keys():
  teams[id] = data[id]['title']

In [7]:
# EDA to get a feeling of how the JSON is structured
columns = []
values = []

for id in data.keys():
  columns = list(data[id]['history'][0].keys())
  values = list(data[id]['history'][0].values())
  break

print(columns)
print(values)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']
['a', 2.40044, 0.145879, 2.40044, 0.145879, {'att': 256, 'def': 23}, {'att': 327, 'def': 29}, 6, 0, 2, 0, 2.9074, 'w', '2019-08-18 18:00:00', 1, 0, 0, 3, 2.2545610000000003]


In [8]:
sevilla_data = []
for row in data['138']['history']:
  sevilla_data.append(list(row.values()))
  
df = pd.DataFrame(sevilla_data, columns=columns)
df.head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,2.40044,0.145879,2.40044,0.145879,"{'att': 256, 'def': 23}","{'att': 327, 'def': 29}",6,0,2,0,2.9074,w,2019-08-18 18:00:00,1,0,0,3,2.254561
1,a,1.39211,0.428332,1.39211,0.428332,"{'att': 181, 'def': 24}","{'att': 238, 'def': 21}",3,6,1,0,2.2710,w,2019-08-23 19:00:00,1,0,0,3,0.963778


In [9]:
# Getting data for all teams
dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))
    
  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  

In [10]:
# Sample check of our newly created DataFrame
dataframes['Barcelona'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,1.03621,0.457129,1.03621,0.457129,"{'att': 113, 'def': 16}","{'att': 345, 'def': 23}",5,3,0,1,1.8714,l,2019-08-16 20:00:00,0,0,1,0,0.579081
1,h,2.85088,0.145785,2.85088,0.145785,"{'att': 229, 'def': 46}","{'att': 358, 'def': 17}",23,0,5,2,2.8905,w,2019-08-25 20:00:00,1,0,0,3,2.705095


### Manipulations to make data as in the original source

In [11]:
for team, df in dataframes.items():
  dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  
# And check how our new dataframes look based on Sevilla dataframe
dataframes['Sevilla'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,...,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,oppda_coef
0,a,2.40044,0.145879,2.40044,0.145879,"{'att': 256, 'def': 23}","{'att': 327, 'def': 29}",6,0,2,...,2.9074,w,2019-08-18 18:00:00,1,0,0,3,2.254561,11.130435,11.275862
1,a,1.39211,0.428332,1.39211,0.428332,"{'att': 181, 'def': 24}","{'att': 238, 'def': 21}",3,6,1,...,2.2710,w,2019-08-23 19:00:00,1,0,0,3,0.963778,7.541667,11.333333


In [12]:
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

In [13]:
frames = []
for team, df in dataframes.items():
  sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
  mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
  final_df = sum_data.join(mean_data)
  final_df['team'] = team
  final_df['matches'] = len(df)
  frames.append(final_df)
  
full_stat = pd.concat(frames)

In [14]:
full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat.sort_values('pts', ascending=False, inplace=True)
full_stat.reset_index(inplace=True, drop=True)
full_stat['position'] = range(1,len(full_stat)+1)

In [15]:
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

In [16]:
cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)

In [17]:
col_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
full_stat = full_stat[col_order]
pd.options.display.float_format = '{:,.2f}'.format
full_stat.head(10)

,position,team,matches,wins,draws,loses,scored,missed,pts,xG,...,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
0,1,Real Madrid,38,26,9,3,70,25,87,72.16,...,33.15,8.15,31.66,32.32,9.26,14.48,334,127,77.69,-9.31
1,2,Barcelona,38,25,7,6,86,38,82,72.01,...,37.82,-0.18,32.62,34.93,8.26,17.96,440,146,73.14,-8.86
2,3,Sevilla,38,19,13,6,54,34,70,55.16,...,36.62,2.62,33.63,16.32,8.41,12.64,238,148,65.70,-4.30
3,4,Atletico Madrid,38,18,16,4,51,27,70,58.70,...,31.52,4.52,28.54,24.20,9.69,10.99,264,162,72.35,2.35
4,5,Villarreal,38,18,6,14,63,49,60,65.61,...,48.59,-0.41,41.83,15.60,10.90,12.07,218,228,64.05,4.05
5,6,Real Sociedad,38,16,8,14,56,48,56,50.92,...,41.89,-6.11,35.20,12.00,8.65,11.92,231,192,57.18,1.18
6,7,Granada,38,16,8,14,52,45,56,45.02,...,43.44,-1.56,39.72,1.58,10.18,7.77,135,197,52.60,-3.40
7,8,Getafe,38,14,12,12,43,37,54,45.19,...,43.02,6.02,38.56,1.32,7.01,6.30,176,119,53.79,-0.21
8,9,Valencia,38,14,11,13,46,53,53,45.58,...,59.07,6.07,52.38,-12.75,14.09,10.54,217,262,45.43,-7.57
9,10,Osasuna,38,13,13,12,46,54,52,49.04,...,49.58,-4.42,45.86,-2.77,9.10,8.51,197,191,51.51,-0.49


## Scraping data for all teams of all leagues of all seasons

Testing the flow before going full into the process

In [18]:
season_data = dict()
season_data[seasons[4]] = full_stat
print(season_data)
full_data = dict()
full_data[leagues[0]] = season_data
print(full_data)

{'2019':     position             team  matches  wins  draws  loses  scored  missed  \
0          1      Real Madrid       38    26      9      3      70      25   
1          2        Barcelona       38    25      7      6      86      38   
2          3          Sevilla       38    19     13      6      54      34   
3          4  Atletico Madrid       38    18     16      4      51      27   
4          5       Villarreal       38    18      6     14      63      49   
5          6    Real Sociedad       38    16      8     14      56      48   
6          7          Granada       38    16      8     14      52      45   
7          8           Getafe       38    14     12     12      43      37   
8          9         Valencia       38    14     11     13      46      53   
9         10          Osasuna       38    13     13     12      46      54   
10        11    Athletic Club       38    13     12     13      41      38   
11        12          Levante       38    14      7    

Combining the prviouas codes to extract all data for all leagues.

In [19]:
full_data = dict()
for league in leagues:
  
  season_data = dict()
  for season in seasons:    
    url = base_url+'/'+league+'/'+season
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    # Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
    scripts = soup.find_all('script')
    
    string_with_json_obj = ''

    # Find data for teams
    for el in scripts:
        if 'teamsData' in el.text:
          string_with_json_obj = el.text.strip()

    # print(string_with_json_obj)

    # strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    
    
    # convert JSON data into Python dictionary
    data = json.loads(json_data)
    
    # Get teams and their relevant ids and put them into separate dictionary
    teams = {}
    for id in data.keys():
      teams[id] = data[id]['title']
      
    # EDA to get a feeling of how the JSON is structured
    # Column names are all the same, so we just use first element
    columns = []
    # Check the sample of values per each column
    values = []
    for id in data.keys():
      columns = list(data[id]['history'][0].keys())
      values = list(data[id]['history'][0].values())
      break
      
    # Getting data for all teams
    dataframes = {}
    for id, team in teams.items():
      teams_data = []
      for row in data[id]['history']:
        teams_data.append(list(row.values()))

      df = pd.DataFrame(teams_data, columns=columns)
      dataframes[team] = df
      # print('Added data for {}.'.format(team))
      
    
    for team, df in dataframes.items():
      dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      
    cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
    cols_to_mean = ['ppda_coef', 'oppda_coef']
    
    frames = []
    for team, df in dataframes.items():
      sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
      mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
      final_df = sum_data.join(mean_data)
      final_df['team'] = team
      final_df['matches'] = len(df)
      frames.append(final_df)

    full_stat = pd.concat(frames)
    
    full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
    full_stat.sort_values('pts', ascending=False, inplace=True)
    full_stat.reset_index(inplace=True, drop=True)
    full_stat['position'] = range(1,len(full_stat)+1)
    
    full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
    full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
    full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']
    
    cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
    full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)
    
    col_order = ['position', 'team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
    full_stat = full_stat[col_order]
    full_stat = full_stat.set_index('position')
    # print(full_stat.head(20))
    
    season_data[season] = full_stat
  
  df_season = pd.concat(season_data)
  full_data[league] = df_season
  
data = pd.concat(full_data)
data.head()
  

team  matches  wins  draws  loses  scored  \
             position                                                         
La_liga 2015 1               Barcelona       38    29      4      5     112   
             2             Real Madrid       38    28      6      4     110   
             3         Atletico Madrid       38    28      4      6      63   
             4              Villarreal       38    18     10     10      44   
             5           Athletic Club       38    18      8     12      58   

                       missed  pts     xG  xG_diff  ...   xGA  xGA_diff  \
             position                               ...                   
La_liga 2015 1             29   91 113.60     1.60  ... 34.03      5.03   
             2             34   90  90.45   -19.55  ... 45.23     11.23   
             3             18   88  54.93    -8.07  ... 27.80      9.80   
             4             35   64  40.49    -3.51  ... 41.63      6.63   
             5             45   62  53.90    -4.10  ... 45.33      0.33   

                       npxGA  npxGD  ppda_coef  oppda_coef  deep  \
             position                                              
La_liga 2015 1         33.29  66.19       6.01       15.06   570   
             2         42.26  41.50       9.25       14.57   404   
             3         27.05  26.39       8.83        9.05   261   
             4         39.40  -1.14       9.92        9.03   188   
             5         40.12   9.92       8.11        9.67   221   

                       deep_allowed  xpts  xpts_diff  
             position                                 
La_liga 2015 1                  163 94.38       3.38  
             2                  211 79.09     -10.91  
             3                  170 72.28     -15.72  
             4                  215 52.11     -11.89  
             5                  174 58.24      -3.76  

[5 rows x 21 columns]

## Exporting data to CSV file

In [20]:
data.to_csv('League_data.csv')